In [16]:
# ---------------------------
# Imports
# ---------------------------
from fastapi import FastAPI, HTTPException
from prometheus_client import Counter, Histogram, generate_latest, CONTENT_TYPE_LATEST, REGISTRY
from fastapi.responses import Response
import httpx
import time
import socket
import threading
import asyncio
import uvicorn

# ---------------------------
# Prometheus Metrics (avoid duplicates)
# ---------------------------
def get_metric(name, metric_type, description, labels):
    """Check if metric exists, otherwise create it"""
    for collector in list(REGISTRY._collector_to_names):
        if name in REGISTRY._collector_to_names[collector]:
            return collector
    if metric_type == "counter":
        return Counter(name, description, labels)
    elif metric_type == "histogram":
        return Histogram(name, description, labels)
    else:
        raise ValueError("Unknown metric type")

REQUEST_COUNT = get_metric("http_requests_total", "counter", "Total HTTP requests", ["method", "endpoint"])
REQUEST_LATENCY = get_metric("http_request_latency_seconds", "histogram", "Request latency", ["endpoint"])

# ---------------------------
# FastAPI App
# ---------------------------
app = FastAPI(title="NetDiag API", description="Network Diagnostics API")

# ---------------------------
# Middleware for Metrics
# ---------------------------
@app.middleware("http")
async def metrics_middleware(request, call_next):
    start = time.time()
    response = await call_next(request)
    REQUEST_COUNT.labels(method=request.method, endpoint=request.url.path).inc()
    REQUEST_LATENCY.labels(endpoint=request.url.path).observe(time.time() - start)
    return response

# ---------------------------
# Endpoints
# ---------------------------
@app.get("/")
def root():
    return {"service": "NetDiag API", "status": "running"}

@app.get("/health")
def health():
    return {"status": "ok"}

@app.get("/ready")
def ready():
    return {"status": "ready"}

@app.get("/metrics")
def metrics():
    return Response(generate_latest(), media_type=CONTENT_TYPE_LATEST)

@app.get("/ping")
def ping(host: str):
    start = time.time()
    try:
        socket.gethostbyname(host)
        latency = round(time.time() - start, 4)
        return {"host": host, "status": "reachable", "latency": latency}
    except socket.error:
        raise HTTPException(status_code=404, detail=f"{host} not reachable")

@app.post("/http_check")
def http_check(url: str):
    start = time.time()
    try:
        r = httpx.get(url, timeout=5)
        latency = round(time.time() - start, 4)
        return {"url": url, "status": r.status_code, "latency": latency}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# ---------------------------
# Run Uvicorn safely in Colab
# ---------------------------
def start_server():
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    asyncio.run(server.serve())

threading.Thread(target=start_server, daemon=True).start()

# ---------------------------
# Test Endpoints
# ---------------------------
import time
time.sleep(3)  # wait a bit for server to start

import httpx
client = httpx.Client(base_url="http://127.0.0.1:8000")

try:
    print("Root:", client.get("/").json())
    print("Health:", client.get("/health").json())
    print("Ready:", client.get("/ready").json())
    print("Ping Google:", client.get("/ping?host=google.com").json())
except Exception as e:
    print("Connection Error:", e)


INFO:     Started server process [695]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:35304 - "GET / HTTP/1.1" 200 OK
Root: {'service': 'NetDiag API', 'status': 'running'}
INFO:     127.0.0.1:35304 - "GET /health HTTP/1.1" 200 OK
Health: {'status': 'ok'}
INFO:     127.0.0.1:35304 - "GET /ready HTTP/1.1" 200 OK
Ready: {'status': 'ready'}
INFO:     127.0.0.1:35304 - "GET /ping?host=google.com HTTP/1.1" 200 OK
Ping Google: {'host': 'google.com', 'status': 'reachable', 'latency': 0.0054}
